In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')
train.head()
X_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [2]:
X_train = train.drop(['Survived'], axis = 1)
y_train = train['Survived']
X_train.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
X_train.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [4]:
X_train.fillna(X_train.mean(), inplace = True)
X_train.isnull().sum()
X_test.fillna(X_test.mean(), inplace = True)
X_test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64

In [5]:
for col in X_train.columns:
    if X_train[col].dtypes == 'object':
        X_train[col] = X_train[col].fillna(X_train[col].value_counts().index[0])
X_train.isnull().sum()
for col in X_test.columns:
    if X_test[col].dtypes == 'object':
        X_test[col] = X_test[col].fillna(X_test[col].value_counts().index[0])
X_test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [6]:
#X_train.drop(['PassengerId', 'Ticket', 'Fare'], axis = 1, inplace = True)
#X_test.drop('Name', axis = 1, inplace = True)

In [7]:
X_train.head()
#X_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,C23 C25 C27,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,C23 C25 C27,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,C23 C25 C27,S


In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
for col in X_train.columns:
    if X_train[col].dtypes == 'object':
        X_train[col] = le.fit_transform(X_train[col])
for col in X_test.columns:
    if X_test[col].dtypes == 'object':
        X_test[col] = le.fit_transform(X_test[col])


In [9]:
from sklearn.tree import DecisionTreeClassifier as DT
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import VotingClassifier

dt = DT(criterion='gini',random_state = 1)

max_depths = np.linspace(1, 32, 32, endpoint=True)
min_samples_leafs = np.linspace(0.1, 0.5, 5, endpoint=True)
param_grid_dt = {'max_depth' : max_depths,'min_samples_leaf' : min_samples_leafs}

lr = LR(random_state=1, solver = 'lbfgs')
c_space = np.logspace(-5,8,15)
param_grid_lr = {'C' : c_space}

knn = KNN()
param_grid_knn = {'n_neighbors' : np.arange(1,50)}

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.fit_transform(X_test)

dt_cv = GridSearchCV(dt, param_grid_dt, cv = 5)
lr_cv = GridSearchCV(lr, param_grid_lr, cv = 5)
knn_cv = GridSearchCV(knn, param_grid_knn, cv = 5)

classifiers = [('Logistic Regression', lr_cv), 
              ('Decision tree Classifier', dt_cv),
              ('K Nearest Neighbors', knn_cv)]

vc = VotingClassifier(estimators = classifiers)
vc.fit(X_train_std, y_train)

D:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
D:\Anaconda\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda\lib\site-packages\sklearn\base.py:464: DataConversionWarning: Data with input dtype int32, int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


VotingClassifier(estimators=[('Logistic Regression', GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=1, ...   pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0))],
         flatten_transform=None, n_jobs=None, voting='hard', weights=None)

In [10]:
vc.score(X_train_std, y_train)

0.8103254769921436

In [11]:
y_pred = vc.predict(X_test_std)

In [13]:
submission = pd.DataFrame({'PassengerId':X_test['PassengerId'],'Survived':y_pred})
submission.head()
submission.to_csv('predictions.csv', index=False)
submission.shape

(418, 2)